# **Example PSI Model Synthesis Notebook**

Tom Schad 

----

- This notebook does an example synthesis of the Global corona using a Predictive Sciences MHD model. 


----

In [103]:
import pycelp 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import os
os.environ["XUVTOP"] = '/usr/local/ssw/packages/chianti/dbase/'
import psi
import sunpy.coordinates.sun as sun
plt.rcParams["image.origin"]='lower'   ## have to worry about flips? 
plt.rcParams["image.interpolation"]='nearest'
from scipy.interpolate import RegularGridInterpolator as rgi
from scipy.interpolate import interp1d

## **Create an instance of the psi.Model class to load and interact with PSI coronal model data**

- Models used here were downloaded using this website from [Predictive Sciences](https://www.predsci.com/hmi/data_access.php)
- We picked the date of the US total solar eclipse (21 Aug 2017) using the med-cor-thermo2 model for this demonstration

In [30]:
## carrington rotation 2194
modelName = 'hmi__med-cor-thermo2-std01__med-hel-poly-std01'
corona = psi.Model('/home/tschad/Dropbox/psiData_21Aug2017_thermo2_12UTC/corona/')

In [3]:
## return the instance name for some basic information on the class
corona

psi Model class
    ---------------------
    Data Directory Names: /home/tschad/Dropbox/psiData_21Aug2017_thermo2_12UTC/corona/
    Number of longitude samples: 181
    Number of latitude samples: 100
    Number of radial samples: 150
    Data shape: (181, 100, 150)
    
    Variables: 
    lons -- Longitudes
    lats -- Latitudes
    rs   -- Radial samples 
    br,bt,bp  -- Spherical components of magnetic field 
    bx,by,bz  -- Cartesian components of magnetic field 
    bmag      -- total magnetic field intensity
    thetaBlocal -- location inclination of magnetic field in solar frame
    

In [68]:
kk = 5

fig,ax = plt.subplots(2,2,figsize = (8,5))
ax = ax.flatten()
ax[0].imshow(corona.bmag[:,:,kk].T*np.cos(corona.thetaBlocal[:,:,kk]).T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[1].imshow(corona.thetaBlocal[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[2].imshow(corona.temp[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[3].imshow(corona.ne[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
labels = 'Magnetogram','Local B Inclination','Temperature','Electron Density'
for n in range(4): ax[n].set_title(labels[n])

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle(modelName + '\nRadial Coordinate: ' + str(corona.rs[kk]))

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This is the GONG 2194_000 synoptic map 

<img src="https://gong.nso.edu/data/magmap/QR/mqj/201708/mrmqj170830/mrmqj170830t0205c2194_000.jpg" />

## **Initialize the pyCELP model of the line or lines to be synthesized**

- Here we start with Fe XIII 1074 nm and keep the model fairly "small" at 50 levels.  The errors incurred by using reduced numbers of atomic levels is addressed in [Schad & Dima (2020)](https://rdcu.be/b5J2X) 


In [5]:
fe13 = pycelp.Ion('fe_13',nlevels = 50)

 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


In [6]:
# should I parallelize these calculations right away? 
print(181*100.*150. * 484e-6 /80.)  ## seconds on 80 cores

16.42575


In [148]:
wvair = 10747. 

totn  = np.zeros_like(corona.br)
rho00 = np.zeros_like(corona.br)
sig   = np.zeros_like(corona.br)

for ii in range(0,181,1): 
    print(ii)
    for jj in range(0,100,1): 
        for kk in range(0,150,1): 
            edens = corona.ne[ii,jj,kk]
            etemp = corona.temp[ii,jj,kk]
            ht = (corona.rs[kk]-1).clip(1.e-8)
            thetab = np.rad2deg(corona.thetaBlocal[ii,jj,kk])
            
            fe13.calc_rho_sym(edens,etemp,ht, thetab)
            upper_lev_rho00 = fe13.get_upper_level_rho00(wvair)
            upper_lev_alignment = fe13.get_upper_level_alignment(wvair)
            total_ion_population = fe13.totn
            
            totn[ii,jj,kk] = total_ion_population
            rho00[ii,jj,kk] = upper_lev_rho00
            sig[ii,jj,kk] = total_ion_population

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


## **Setup forward synthesis geometry**

In [149]:
crn = 2194.1  ## not yet sure what to use here?  How to get fractional carrington rotation of the PSI data?
crt        = sun.carrington_rotation_time(crn)
print('Carrington Rotation Time: ',crt)
obsLon     = sun.L0(crt).deg
b0         = sun.B0(crt).deg
Obs_Sun_AU = sun.earth_distance(crt).value
rsunarc    = sun.angular_radius(crt).value  ## radius of sun in arcseconds .. later replace with sun ephemeris
fov_arc    = 6.*960.  ## +/- 3 rsun
arcsamp    = 20. ## sampling in arcsecond
print(obsLon,b0)
print(Obs_Sun_AU,rsunarc,fov_arc,arcsamp)

Carrington Rotation Time:  2017-08-19 04:44:52.946
323.9999982980418 6.8306805033001865
1.012050123894962 947.8098777838427 5760.0 20.0


In [150]:
##################
## recall the simulation geometry and plot
## compare to GONG maps of the CR 2189
## rObs is the radial position of the observer in solar radii
rObs     = Obs_Sun_AU * (1.495978707e11/6.96340e8)   ##  rObs in solar radii?
thetaObs = np.pi/2. - np.deg2rad(b0)
phiObs   = np.deg2rad(obsLon)
xObs,yObs,zObs = rObs*np.sin(thetaObs)*np.cos(phiObs),rObs*np.sin(thetaObs)*np.sin(phiObs),rObs*np.cos(thetaObs)
print(xObs,yObs,zObs)

174.65060561060434 -126.89110044448852 25.859384238598253


In [151]:
## setup the synthesized field of view
yarc = np.linspace(-fov_arc/2.,fov_arc/2.,int(np.ceil(fov_arc/arcsamp)))
zarc = yarc
yya,zza  = np.meshgrid(yarc,zarc,indexing = 'ij')
rra = np.sqrt(yya**2. + zza**2.)
m_behind_sun = 1.*(rra>rsunarc)

In [152]:
plt.figure()
plt.imshow(m_behind_sun)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [153]:
def euler_ry(alpha):
    '''Euler rotation matrix about y axis '''
    ry = np.array([[ np.cos(alpha), 0., np.sin(alpha)],
                   [            0., 1., 0.],
                   [-np.sin(alpha), 0., np.cos(alpha)]])
    return ry

def euler_rz(alpha):
    '''Euler rotation matrix about z axis '''
    rz = np.array([[ np.cos(alpha), -np.sin(alpha), 0.],
                   [ np.sin(alpha),  np.cos(alpha), 0.],
                   [            0.,             0., 1.]])
    return rz

################
## define new coordinate system with x towards
## observer, and z towards north pole
## get points in the plane of the sky

xxObs = np.zeros_like(yya)
yyObs = rObs * np.tan(np.deg2rad(yya/3600.))
zzObs = rObs * np.tan(np.deg2rad(zza/3600.))

## rotate these points into the model geometry with Euler rotation
rotm = euler_rz(-phiObs) @ euler_ry(-(thetaObs-np.pi/2.))
xyz_model = rotm.T  @ np.stack((xxObs.flatten(),yyObs.flatten(),zzObs.flatten()))

#plt.figure()
#plt.imshow(xyz_model[1,:].reshape(xxObs.shape).T,origin = 'lower')

## NOW WITH THE XYZ_MODEL POINTS AND THE LOCATION OF THE OBSERVER,
## come up with the parametric equations for the los of sight
## and then get the spherical coordinates, interpolate for rho and temps, etc.
## synthesize and integrate for integrated I,Q,U, (V?)

losvec = np.stack((xObs-xyz_model[0,:],yObs-xyz_model[1,:],zObs - xyz_model[2,:]))
losveclen = np.linalg.norm(losvec,axis=0,ord=2,keepdims = True)
losvec = losvec / losveclen
startpt = np.stack((xyz_model[0,:],xyz_model[1,:],xyz_model[2,:]))

In [154]:
startpt.shape,losvec.shape

((3, 82944), (3, 82944))

In [155]:
## TO DO!! 
bx   = corona.bx
by   = corona.by
bz   = corona.bz

lons,lats,rs = corona.lons,corona.lats,corona.rs

#### Get interpolating functions for all necessary simulation data
totnInt = rgi((lons,lats,rs),totn,method = 'linear',fill_value = 0.,bounds_error = False)
rhoInt  = rgi((lons,lats,rs),rho00,method = 'linear',fill_value = 0.,bounds_error = False)
sigInt  = rgi((lons,lats,rs),sig,method = 'linear',fill_value = 0.,bounds_error = False)
bxInt   = rgi((lons,lats,rs),bx,method = 'linear',fill_value = 0.,bounds_error = False)
byInt   = rgi((lons,lats,rs),by,method = 'linear',fill_value = 0.,bounds_error = False)
bzInt   = rgi((lons,lats,rs),bz,method = 'linear',fill_value = 0.,bounds_error = False)

In [156]:
steps = np.linspace(-3,3,int(np.ceil(6./(10./960.))))  ## radii units

In [157]:
## probably should get all points and interpolate for the values once?  
## memory efficiency? 
## ...actually below it is one step at a time along the line-of-sight vectors..that may be okay

In [158]:
qnj,A_ein,hnu,D_coeff,E_coeff,geff = 1,14.,1,1,1,1.5

## SYNTHESIS
totIQU = np.zeros((xxObs.shape[0],xxObs.shape[1],4))
steps = np.linspace(-3,3,np.int(np.ceil(6./(10./960.))))  ## radii units
for nstep,losstep in enumerate(steps):
    print(nstep,len(steps))

    ## get points along line of sight in cartesian and spherical coords
    xyz_next = startpt + losvec*losstep
    rm = np.sqrt(np.sum(xyz_next**2,axis = 0))
    tm = np.arccos(xyz_next[2,:]/rm)
    pm = np.arctan2(xyz_next[1,:],xyz_next[0,:])
    pm[pm<0] += 2*np.pi
    flat = np.array([pm,tm,rm])
    ## get ThetaB (inclination of B wrt to LOS)
    bxyzs = np.stack((bxInt(flat.T),byInt(flat.T),bzInt(flat.T)))
    blens = np.linalg.norm(bxyzs,axis =0)

    ## losvec len is 1
    thetaBlos = np.arccos((np.sum(bxyzs*losvec,axis=0)/blens).clip(min = -1,max=1))
    thetaBlos[np.isnan(thetaBlos)] = 0.

    ## angle between losvec and the vector from disk center
    rlens = np.linalg.norm(xyz_next,axis=0)
    thetaDClos = np.arccos((np.sum(xyz_next*losvec,axis=0)/rlens).clip(min = -1,max=1))

    ## get projection of B onto plane perpendicular to LOS
    ## and the projection of DC vector onto same plane
    Bperp  = bxyzs -  (blens*np.cos(thetaBlos))*losvec
    DCperp = xyz_next - (rlens*np.cos(thetaDClos))*losvec

    ## find angle between Bperp and DCperp, which is the azimuthal angle relative to disk center
    costhetaAzi = np.sum((Bperp*DCperp),axis=0)/(np.linalg.norm(Bperp,axis = 0)*np.linalg.norm(DCperp,axis = 0))
    thetaAzi = np.arccos(costhetaAzi.clip(min =-1,max=1))
    thetaAzi[np.isnan(thetaAzi)] = 0.

    ## total population and atomic alignment
    C_coeff = totnInt(flat.T)*np.sqrt(2.*qnj+1.)*rhoInt(flat.T)*A_ein*hnu/(4.*np.pi)
    sigma   = sigInt(flat.T)

    ## STOKES COEFFICIENTS
    epsI   = C_coeff*(1.0+(1./(2.*np.sqrt(2.)))*(3.*np.cos(thetaBlos)**2 - 1.)*D_coeff*sigma)
    epsQnr = C_coeff*(3./(2.*np.sqrt(2.)))*(np.sin(thetaBlos)**2)*D_coeff*sigma
    epsQ   = np.cos(2.*thetaAzi)*epsQnr
    epsU   = -np.sin(2.*thetaAzi)*epsQnr
    epsV   = C_coeff*np.cos(thetaBlos)*(1399612.2*blens)*(geff + E_coeff*sigma)
    ## is it behind the Sun?

    if (losstep < 0):
        epsI = epsI.reshape(xxObs.shape)*m_behind_sun
        epsQ = epsQ.reshape(xxObs.shape)*m_behind_sun
        epsU = epsU.reshape(xxObs.shape)*m_behind_sun
        epsV = epsV.reshape(xxObs.shape)*m_behind_sun
    else:
        epsI = epsI.reshape(xxObs.shape)
        epsQ = epsQ.reshape(xxObs.shape)
        epsU = epsU.reshape(xxObs.shape)
        epsV = epsV.reshape(xxObs.shape)
    ###

    totIQU[:,:,0] += epsI
    totIQU[:,:,1] += epsQ
    totIQU[:,:,2] += epsU
    totIQU[:,:,3] += epsV

0 576


/tmp/ipykernel_128951/480023948.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  steps = np.linspace(-3,3,np.int(np.ceil(6./(10./960.))))  ## radii units


1 576
2 576
3 576
4 576
5 576
6 576
7 576
8 576
9 576
10 576
11 576
12 576
13 576
14 576
15 576
16 576
17 576
18 576
19 576
20 576
21 576
22 576
23 576
24 576
25 576
26 576
27 576
28 576
29 576
30 576
31 576
32 576
33 576
34 576
35 576
36 576
37 576
38 576
39 576
40 576
41 576
42 576
43 576
44 576
45 576
46 576
47 576
48 576
49 576
50 576
51 576
52 576
53 576
54 576
55 576
56 576
57 576
58 576
59 576
60 576
61 576
62 576
63 576
64 576
65 576
66 576
67 576
68 576
69 576
70 576
71 576
72 576
73 576
74 576
75 576
76 576
77 576
78 576
79 576
80 576
81 576
82 576
83 576
84 576
85 576
86 576
87 576
88 576
89 576
90 576
91 576
92 576
93 576
94 576
95 576
96 576
97 576
98 576
99 576
100 576
101 576
102 576
103 576
104 576
105 576
106 576
107 576
108 576
109 576
110 576
111 576
112 576
113 576
114 576
115 576
116 576
117 576
118 576
119 576
120 576
121 576
122 576
123 576
124 576
125 576
126 576
127 576
128 576
129 576
130 576
131 576
132 576
133 576
134 576
135 576
136 576
137 576
138 576
139 

/tmp/ipykernel_128951/480023948.py:21: RuntimeWarning: invalid value encountered in true_divide
  thetaBlos = np.arccos((np.sum(bxyzs*losvec,axis=0)/blens).clip(min = -1,max=1))
/tmp/ipykernel_128951/480023948.py:34: RuntimeWarning: invalid value encountered in true_divide
  costhetaAzi = np.sum((Bperp*DCperp),axis=0)/(np.linalg.norm(Bperp,axis = 0)*np.linalg.norm(DCperp,axis = 0))


194 576
195 576
196 576
197 576
198 576
199 576
200 576
201 576
202 576
203 576
204 576
205 576
206 576
207 576
208 576
209 576
210 576
211 576
212 576
213 576
214 576
215 576
216 576
217 576
218 576
219 576
220 576
221 576
222 576
223 576
224 576
225 576
226 576
227 576
228 576
229 576
230 576
231 576
232 576
233 576
234 576
235 576
236 576
237 576
238 576
239 576
240 576
241 576
242 576
243 576
244 576
245 576
246 576
247 576
248 576
249 576
250 576
251 576
252 576
253 576
254 576
255 576
256 576
257 576
258 576
259 576
260 576
261 576
262 576
263 576
264 576
265 576
266 576
267 576
268 576
269 576
270 576
271 576
272 576
273 576
274 576
275 576
276 576
277 576
278 576
279 576
280 576
281 576
282 576
283 576
284 576
285 576
286 576
287 576
288 576
289 576
290 576
291 576
292 576
293 576
294 576
295 576
296 576
297 576
298 576
299 576
300 576
301 576
302 576
303 576
304 576
305 576
306 576
307 576
308 576
309 576
310 576
311 576
312 576
313 576
314 576
315 576
316 576
317 576
318 576


In [159]:
plt.figure()
plt.imshow(np.log10(totIQU[:,:,0]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_128951/1191100609.py:2: RuntimeWarning: invalid value encountered in log10
  plt.imshow(np.log10(totIQU[:,:,0]))


In [ ]:
## generating polarized spectra